In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold,RandomizedSearchCV

import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import RandomOverSampler
import pickle
import joblib




In [2]:
data=pd.read_csv("tickets.csv")

C:\Users\sridhar\AppData\Local\Temp\ipykernel_3832\1152863127.py:1: DtypeWarning: Columns (10,30,33,34,44,55,57,64) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("tickets.csv")


In [3]:
data

,AD OU Location,Affected User Business Unit,Affected User Company,Affected User Country,Affected User Location,Affected User Region,Alert Asset Name,Application Request,Approval Status,Approval Status.1,...,Subject,Survey Result,Tags,Type,Time Tracked,Last Updated Time,Updated By,Urgency,Workflow Field,Zip_Postal Code
0,"OU=Users,OU=GMC,OU=Mexico,OU=AAM-Global",Driveline,American Axle & Manufacturing Inc,Mexico,Guanajuato Manufacturing - Plant 5,North America,NaN,NaN,NaN,Not Requested,...,GMC 5 - Network Account - Unlock Account,NaN,Edward Acuna,Incident,0:00:00,11/12/2023 1:35,Edward Acuna,Low,NaN,36100
1,"OU=Users,OU=PNE,OU=Asia,OU=AAM-Global",Corporate - India,American Axle & Manufacturing Inc,India,Pune Business Office & Engineering Center,India,NaN,NaN,NaN,Not Requested,...,Computer Accessories Request Bundle for Chetan...,NaN,"Computer Accessories Request,Sunil Dhebe",Service Request,0:00:00,11/12/2023 0:35,Sunil Dhebe,Low,19000251677,411014
2,"OU=Users,OU=PNE,OU=Asia,OU=AAM-Global",Corporate - India,American Axle & Manufacturing Inc,India,Pune Business Office & Engineering Center,India,NaN,NaN,NaN,Not Requested,...,Need to add blue flag to BI in PLM.,NaN,"Sunil Dhebe,Vishvanath Kadam,Release/Unrelease",Incident,0:00:00,11/18/2023 7:35,Vishvanath Kadam,Low,NaN,411014
3,"OU=Users,OU=ZEL,OU=Europe,OU=AAM-Global",Metal Forming,NaN,Germany,Zell Manufacturing Facility,Europe,NaN,NaN,NaN,Not Requested,...,Remote Verbindung zum Pusher ist nicht in Funk...,NaN,"Ieisha Alexander,Hans-Juergen Schenk,Christoph...",Service Request,0:00:00,11/13/2023 9:36,Christoph Roessel,Low,NaN,77736
4,"OU=Users,OU=ZEL,OU=Europe,OU=AAM-Global",Metal Forming,American Axle & Manufacturing Inc,Germany,Zell Manufacturing Facility,Europe,NaN,NaN,NaN,Not Requested,...,Q-Studio läuft nicht mehr,NaN,"Oliver Kurz,Hans-Juergen Schenk",Incident,0:00:00,11/12/2023 8:35,Hans-Juergen Schenk,Low,NaN,77736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8587,"OU=Users,OU=PUN,OU=Asia,OU=AAM-Global",Driveline,American Axle & Manufacturing Inc,India,Pune Manufacturing Facility,India,NaN,NaN,Requested,Not Requested,...,Request for Rahul Patil : Siemens Teamcenter V...,NaN,NaN,Service Request,0:00:00,12/7/2023 0:27,Rahul Patil,Low,Engineering Application Request,414301
8588,"OU=Users,OU=OSL,OU=Europe,OU=AAM-Global",Metal Forming,American Axle & Manufacturing Inc,Czech Republic,Oslavany Manufacturing Facility,Europe,NaN,NaN,NaN,Not Requested,...,Dorda Petr (61071),NaN,NaN,Incident,0:00:00,12/7/2023 0:38,Petr Kocanda,Low,NaN,664 12
8589,"OU=Users,OU=HAU,OU=Europe,OU=AAM-Global",Metal Forming,American Axle & Manufacturing Inc,Germany,Hausach Manufacturing Facility,Europe,NaN,NaN,NaN,Not Requested,...,Lohnschein Drucken geht nicht,NaN,Jackie Robinson,Incident,0:00:00,12/7/2023 1:03,Jackie Robinson,Low,NaN,NaN
8590,"OU=Users,OU=PNE,OU=Asia,OU=AAM-Global",Corporate - India,American Axle & Manufacturing Inc,India,Pune Business Office & Engineering Center,India,NaN,NaN,NaN,Not Requested,...,Reimbursement request not visible,NaN,NaN,Incident,0:00:00,12/7/2023 1:08,Harshal Sonawane,Low,NaN,411014


In [4]:
data.head()

,AD OU Location,Affected User Business Unit,Affected User Company,Affected User Country,Affected User Location,Affected User Region,Alert Asset Name,Application Request,Approval Status,Approval Status.1,...,Subject,Survey Result,Tags,Type,Time Tracked,Last Updated Time,Updated By,Urgency,Workflow Field,Zip_Postal Code
0,"OU=Users,OU=GMC,OU=Mexico,OU=AAM-Global",Driveline,American Axle & Manufacturing Inc,Mexico,Guanajuato Manufacturing - Plant 5,North America,NaN,NaN,NaN,Not Requested,...,GMC 5 - Network Account - Unlock Account,NaN,Edward Acuna,Incident,0:00:00,11/12/2023 1:35,Edward Acuna,Low,NaN,36100
1,"OU=Users,OU=PNE,OU=Asia,OU=AAM-Global",Corporate - India,American Axle & Manufacturing Inc,India,Pune Business Office & Engineering Center,India,NaN,NaN,NaN,Not Requested,...,Computer Accessories Request Bundle for Chetan...,NaN,"Computer Accessories Request,Sunil Dhebe",Service Request,0:00:00,11/12/2023 0:35,Sunil Dhebe,Low,19000251677,411014
2,"OU=Users,OU=PNE,OU=Asia,OU=AAM-Global",Corporate - India,American Axle & Manufacturing Inc,India,Pune Business Office & Engineering Center,India,NaN,NaN,NaN,Not Requested,...,Need to add blue flag to BI in PLM.,NaN,"Sunil Dhebe,Vishvanath Kadam,Release/Unrelease",Incident,0:00:00,11/18/2023 7:35,Vishvanath Kadam,Low,NaN,411014
3,"OU=Users,OU=ZEL,OU=Europe,OU=AAM-Global",Metal Forming,NaN,Germany,Zell Manufacturing Facility,Europe,NaN,NaN,NaN,Not Requested,...,Remote Verbindung zum Pusher ist nicht in Funk...,NaN,"Ieisha Alexander,Hans-Juergen Schenk,Christoph...",Service Request,0:00:00,11/13/2023 9:36,Christoph Roessel,Low,NaN,77736
4,"OU=Users,OU=ZEL,OU=Europe,OU=AAM-Global",Metal Forming,American Axle & Manufacturing Inc,Germany,Zell Manufacturing Facility,Europe,NaN,NaN,NaN,Not Requested,...,Q-Studio läuft nicht mehr,NaN,"Oliver Kurz,Hans-Juergen Schenk",Incident,0:00:00,11/12/2023 8:35,Hans-Juergen Schenk,Low,NaN,77736


In [141]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8592 entries, 0 to 8591
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Combined_Text  8592 non-null   object
 1   Group          8592 non-null   object
dtypes: object(2)
memory usage: 134.4+ KB


###observing the data types of the columns present

In [140]:
for column in data.columns:
    print(f"Column: {column}, Data Type: {data[column].dtype}")


Column: Combined_Text, Data Type: object
Column: Group, Data Type: object


### downloading all the word pre-processing techniques

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sridhar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sridhar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sridhar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data.head()

### Determining the text columns which are closely related to the target variable called group

In [6]:
text_columns = ['Description', 'Category', 'Priority', 'Affected User Business Unit', 'Department', 'Employee Type', 'Source', 'Status','Affected User Location']

### combining the text columns and removing all the punctuations

In [7]:
data['Combined_Text'] = data[text_columns].apply(lambda row: ' '.join([str(row[col]) for col in text_columns]), axis=1)

### here we are ready to use the combined_text as training and Group as testing data and combined text is the combination of all the independent variables

In [8]:
relevant_columns = ['Combined_Text', 'Group']

In [9]:
print("the relevant columns taken for group prediction:",relevant_columns)

the relevant columns taken for group prediction: ['Combined_Text', 'Group']


In [10]:
print("the combined text is:",data['Combined_Text'] )

the combined text is: 0       Unlock Account Network Low Driveline 8.0" Fron...
1       nan nan Low Corporate - India Product Engineer...
2       Team, Please add blue flag to below BI's. Thes...
3       nan nan Low Metal Forming Information Technolo...
4       Q-Studio bringt Fehlermeldung: "Q-Studio Infor...
                              ...                        
8587    Please use the "Engineering Application Reques...
8588    Dorda Petr (61071) nefunk?ní heslo do plex nan...
8589    Ich kann kein Lohnschein ausdrucken. Grund im ...
8590    oracle expense request not visible. kindly res...
8591    nan nan Low Driveline Supply Chain Management ...
Name: Combined_Text, Length: 8592, dtype: object


In [10]:
#now dropping rows with missing values

In [11]:
data=data[relevant_columns].dropna(subset=['Combined_Text', 'Group'])

In [12]:
data

,Combined_Text,Group
0,"Unlock Account Network Low Driveline 8.0"" Fron...",GMC - IT Site Support
1,nan nan Low Corporate - India Product Engineer...,India - Service Desk
2,"Team, Please add blue flag to below BI's. Thes...",ENG - PLM Support
3,nan nan Low Metal Forming Information Technolo...,ZGMF - IT Site Support
4,"Q-Studio bringt Fehlermeldung: ""Q-Studio Infor...",ZGMF - IT Site Support
...,...,...
8587,"Please use the ""Engineering Application Reques...",No Group
8588,Dorda Petr (61071) nefunk?ní heslo do plex nan...,Europe - Service Desk
8589,Ich kann kein Lohnschein ausdrucken. Grund im ...,HGMF - IT Site Support
8590,oracle expense request not visible. kindly res...,India - Service Desk


### applying the most important process of converting words into it's simple root form for the model to understand the words easily

In [13]:
lemmatizer = WordNetLemmatizer()

### printing to check whether lemmatizer is applied successfully or not

In [14]:
lemmatizer

<WordNetLemmatizer>

### from the above lemmatization applying this lemmatization to all of the combined independent variables closely related to the target variable iterating through each word and applying lemmatization process

In [15]:
data['Combined_Text'] = data['Combined_Text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(str(x))]))

### observing the number of rows in the dataset using the shape function

In [16]:
data.shape

(8592, 2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [ ]:
count_vectorizer = CountVectorizer()
X_train_bow = count_vectorizer.fit_transform(X_train)
X_test_bow = count_vectorizer.transform(X_test)

In [ ]:
logistic_regression_classifier = LogisticRegression()
logistic_regression_classifier.fit(X_train_bow, y_train)

In [ ]:
y_pred = logistic_regression_classifier.predict(X_test_bow)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_rep)
print("\nConfusion Matrix:\n", conf_matrix)

In [ ]:
new_ticket_data = {
    'Description': "User reporting login issue",
    'Category': "Login",
    'Priority': "High",
    'Affected User Business Unit': "IT",
    'Department': "Support",
    'Employee Type': "Full-time",
    'Source': "Web",
    'Status': "Open",
    'Affected User Location':"europe Manufacturing Facility"
    
}

In [ ]:
new_ticket_text = ' '.join([str(new_ticket_data[col]) for col in text_columns])
new_ticket_bow = count_vectorizer.transform([new_ticket_text])
predicted_team = logistic_regression_classifier.predict(new_ticket_bow)

In [ ]:
predicted_team_category = label_encoder.inverse_transform(predicted_team)

In [ ]:
predicted_team_category

In [ ]:
#11th december

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data.info()

In [ ]:
test_size = 0.3

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(data['Combined_Text'],data['Group'],test_size = test_size,random_state=42)

In [ ]:
X_train,X_test,y_train,y_test

In [ ]:
count_vectorizer_for_random = CountVectorizer()

In [ ]:
X_train_random = count_vectorizer_for_random.fit_transform(X_train)

In [ ]:
X_test_random = count_vectorizer_for_random.transform(X_test)

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42)

In [ ]:
rf_classifier

In [ ]:
rf_classifier.fit(X_train_random,y_train)

In [ ]:
y_prediction_random = rf_classifier.predict(X_test_random)

In [ ]:
y_prediction_random

In [ ]:
accuracy_rf = accuracy_score(y_test, y_prediction_random)

In [ ]:
print("accuracy is:",accuracy_rf)

In [ ]:
new_ticket_data = {
    'Description': "User reporting login issue",
    'Category': "Login",
    'Priority': "High",
    'Affected User Business Unit': "IT",
    'Department': "Support",
    'Employee Type': "Full-time",
    'Source': "Web",
    'Status': "Open",
    'Affected User Location':"europe Manufacturing Facility"
    
}

In [ ]:
new_ticket_text = ' '.join([str(new_ticket_data[col]) for col in text_columns])
new_ticket_bow = count_vectorizer_for_random.transform([new_ticket_text])
predicted_team = rf_classifier.predict(new_ticket_bow)

In [ ]:
new_ticket_text

In [ ]:
new_ticket_bow

In [ ]:
predicted_team

In [ ]:
#using xgboost

In [ ]:
!pip install xgboost nltk

In [ ]:
from xgboost import XGBClassifier

In [ ]:
test_size = 0.3

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(data['Combined_Text'],data['Group'],test_size = test_size, random_state=42)

In [ ]:
count_vectorizer = CountVectorizer()

In [ ]:
X_train_xg = count_vectorizer.fit_transform(X_train)


In [ ]:
X_test_xg = count_vectorizer.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
y_train_encoded = label_encoder.fit_transform(y_train)

In [ ]:
xgb_classifier = XGBClassifier()

In [ ]:
xgb_classifier.fit(X_train_xg, y_train_encoded)

In [ ]:
y_pred_encoded = xgb_classifier.predict(X_test_xg)

In [ ]:
y_pred_encoded

In [ ]:
y_pred_original = label_encoder.inverse_transform(y_pred_encoded)

In [ ]:
y_pred_original

In [ ]:
accuracy = accuracy_score(y_test, y_pred_original)
classification_rep = classification_report(y_test, y_pred_original)
conf_matrix = confusion_matrix(y_test, y_pred_original)


In [ ]:
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_rep)
print("\nConfusion Matrix:\n", conf_matrix)

In [ ]:
new_ticket_data = {
    'Description': "User reporting login issue",
    'Category': "Login",
    'Priority': "High",
    'Affected User Business Unit': "IT",
    'Department': "Support",
    'Employee Type': "Full-time",
    'Source': "Web",
    'Status': "Open",
    'Affected User Location':"europe Manufacturing Facility"
    
}

In [ ]:
new_ticket_text = ' '.join([str(new_ticket_data[col]) for col in text_columns])
new_ticket_bow = count_vectorizer.transform([new_ticket_text])
predicted_team = xgb_classifier.predict(new_ticket_bow)

In [ ]:
predicted_team_final = label_encoder.inverse_transform(predicted_team)

In [ ]:
print("predicted_team_final is:",predicted_team_final)

In [ ]:
#word2vec

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
label_encoder

In [ ]:
data['Group'] = label_encoder.fit_transform(data['Group'])

In [ ]:
data['Group']

In [ ]:
test_size = 0.3

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(data['Combined_Text'],data['Group'],test_size=test_size,random_state=42)

In [ ]:
word2vec_model = Word2Vec(sentences=X_train.apply(word_tokenize), vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
word2vec_model

In [ ]:
def average_word2vec(text, model, vector_size):
    words = word_tokenize(str(text))
    vectors = [model.wv[word] for word in words if word in model.wv]
    if not vectors:
        return [0] * vector_size
    return sum(vectors) / len(vectors)

In [ ]:
X_train_w2v = X_train.apply(lambda x: average_word2vec(x, word2vec_model, vector_size=100))
X_test_w2v = X_test.apply(lambda x: average_word2vec(x, word2vec_model, vector_size=100))

In [ ]:
X_train_w2v

In [ ]:
X_test_w2v

In [ ]:
logistic_regression_classifier = LogisticRegression()

In [ ]:
logistic_regression_classifier

In [ ]:
logistic_regression_classifier.fit(list(X_train_w2v), y_train)

In [ ]:
y_pred = logistic_regression_classifier.predict(list(X_test_w2v))

In [ ]:
y_pred

In [ ]:
accuracy = accuracy_score(y_test, y_pred)


In [ ]:
print("Accuracy:", accuracy)

In [ ]:
new_ticket_data = {
    'Description': "User reporting login issue",
    'Category': "Login",
    'Priority': "High",
    'Affected User Business Unit': "IT",
    'Department': "Support",
    'Employee Type': "Full-time",
    'Source': "Web",
    'Status': "Open",
    'Affected User Location':"europe Manufacturing Facility"
    
}

In [ ]:
new_ticket_text = ' '.join([str(new_ticket_data[col]) for col in text_columns])
new_ticket_w2v = average_word2vec(new_ticket_text, word2vec_model, vector_size=100)
predicted_team = logistic_regression_classifier.predict([new_ticket_w2v])

In [ ]:
predicted_team_category = label_encoder.inverse_transform(predicted_team)

In [ ]:
print("predicted:",predicted_team_category[0])

In [ ]:
#xgboost fine tuning

In [ ]:
#performing xgboost with tfidf vectorization

In [ ]:
test_size = 0.3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['Combined_Text'], data['Group'], test_size=test_size, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test

In [ ]:
tfidf_vectorizer = TfidfVectorizer()


In [ ]:
 X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)


In [ ]:
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [ ]:
xgb_classifier = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=3, reg_alpha=1, reg_lambda=1, n_jobs=-1)

In [ ]:
xgb_classifier

In [ ]:
xgb_classifier.fit(X_train_tfidf, y_train_encoded)

In [ ]:
y_pred_encoded = xgb_classifier.predict(X_test_tfidf)

In [ ]:
print("",y_pred_encoded)

In [ ]:
y_pred_original = label_encoder.inverse_transform(y_pred_encoded)

In [ ]:
y_pred_original

In [ ]:
accuracy = accuracy_score(y_test, y_pred_original)
classification_rep = classification_report(y_test, y_pred_original)
conf_matrix = confusion_matrix(y_test, y_pred_original)

In [ ]:
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_rep)
print("\nConfusion Matrix:\n", conf_matrix)

In [ ]:
new_ticket_data = {
    'Description': "User reporting login issue",
    'Category': "Login",
    'Priority': "High",
    'Affected User Business Unit': "IT",
    'Department': "Support",
    'Employee Type': "Full-time",
    'Source': "Web",
    'Status': "Open",
    'Affected User Location':"europe Manufacturing Facility"
    
}

In [ ]:
new_ticket_text = ' '.join([str(new_ticket_data[col]) for col in text_columns])
new_ticket_doi = tfidf_vectorizer.transform([new_ticket_text])
predicted_team = xgb_classifier.predict(new_ticket_doi)

In [ ]:
predicted_category = label_encoder.inverse_transform(predicted_team)

In [ ]:
print("the predicted group is:",predicted_category[0])

### please scroll down for the main prediction(xgboost+tfidf)which includes accuracy of the entire model and accuracy on unseen data

### hyer-paramter training with tfidf and xgboost

In [17]:
oversampler = RandomOverSampler(random_state=42)

### applying oversampler to increase the data points for the y_train and y_test to be equal in size

In [18]:
oversampler

RandomOverSampler(random_state=42)

In [19]:
X_resampled, y_resampled = oversampler.fit_resample(data['Combined_Text'].values.reshape(-1, 1), data['Group'])

In [20]:
test_size = 0.3

### Splitting the Resampled Data for Training and Testing

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled.flatten(), y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)


### applying union of y_train and y_test

In [22]:
all_classes = set(y_train) | set(y_test)

### Ensuring Consistent Classes in Training and Testing Sets

In [23]:
y_train = y_train[y_train.isin(all_classes)]
y_test = y_test[y_test.isin(all_classes)]


In [24]:
missing_classes = set(y_train) - set(y_test)

In [25]:
y_test = pd.concat([y_test, pd.Series(list(missing_classes))], ignore_index=True)

### applying label encoder to convert text to numerics of both the y_train and Y-test

In [26]:
label_encoder = LabelEncoder()
all_labels = pd.concat([y_train, y_test], axis=0)  
y_all_encoded = label_encoder.fit_transform(all_labels)


###converting the concatenated y values to it's training and testing data

In [27]:
y_train_encoded = y_all_encoded[:len(y_train)]
y_test_encoded = y_all_encoded[len(y_train):]

### tfidf vectorization

In [28]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

### declaring xg boost classification for prediction of the tfidf vectorized data

In [29]:
xgb_classifier = XGBClassifier()

### declaring essential hyper parameters for improving model's accuracy

In [30]:
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 200, 300],
    
}

### printing the hyper-paramter models

In [31]:
print("the paramter grids are:",param_grid)

the paramter grids are: {'max_depth': [3, 4, 5], 'learning_rate': [0.1, 0.01, 0.05], 'n_estimators': [100, 200, 300]}


### Randomized Search Cross Validation for Hyperparameter Tuning in XGBoost Classifier 

In [32]:
random_search = RandomizedSearchCV(
    xgb_classifier,
    param_distributions=param_grid,
    n_iter=20,  
    scoring='accuracy',
    n_jobs=-1,
    cv=3,
    verbose=1,
    random_state=42
)

### fitting the model for further evaluation

In [33]:
random_search.fit(X_train_tfidf,y_train_encoded)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.01, 0.05],
                                        'max_depth': [3, 4, 5],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='accuracy', verbose=1)

In [34]:
best_params = random_search.best_params_

In [74]:
best_params

{'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.1}

### saving the model in an joblib file

In [75]:
joblib.dump(random_search, "xgboost_text_classifier_model.joblib")

['xgboost_text_classifier_model.joblib']

In [76]:
loaded_model = joblib.load("xgboost_text_classifier_model.joblib")

In [78]:
loaded_model

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.01, 0.05],
                                        'max_depth': [3, 4, 5],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, scoring='accuracy', verbose=1)

### prediction process takes place here

In [79]:
y_pred_encoded = loaded_model.predict(X_test_tfidf)

In [110]:
y_pred_encoded

array([37, 68, 60, ..., 52, 68, 25], dtype=int64)

### applying inverse transform for original prediction(conversion of numeric into words)

In [102]:
y_pred_original = label_encoder.inverse_transform(y_pred_encoded)

In [111]:
y_pred_original

array(['CHMF - IT Site Support', 'FIN - IT Finance Analysts',
       'ENG - NVH Support', ..., 'EHQ - IT Site Support',
       'FIN - IT Finance Analysts', 'APP - RPA Support'], dtype=object)

In [112]:
accuracy = accuracy_score(y_test, y_pred_original)

### printing the model's accuracy 

In [113]:
accuracy

0.899535216303182

### The best paramters used

In [114]:
print("Best Parameters:", best_params)

Best Parameters: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.1}


### testing the model on new ticket data

In [230]:
new_ticket_data = {
    'Description': "User reporting login issue",
    'Category': "Login",
    'Priority': "Low",
    'Affected User Business Unit': "IT",
    'Department': "Support",
    'Employee Type': "Full-time",
    'Source': "Web",
    'Status': "Open",
    'Affected User Location': "chennai Manufacturing Facility"
}

In [231]:
new_ticket_text = ' '.join([str(new_ticket_data[col]) for col in text_columns])
new_ticket_text = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(new_ticket_text)])
new_ticket_tfidf = tfidf_vectorizer.transform([new_ticket_text])


In [232]:
new_ticket_pred_encoded = loaded_model.predict(new_ticket_tfidf)

In [233]:
new_ticket_pred_original = label_encoder.inverse_transform(new_ticket_pred_encoded)


###final prediction of the above new ticket data for testing

In [234]:
print("Prediction for the new ticket data:", new_ticket_pred_original[0])

Prediction for the new ticket data: AAM - L2 Service Desk


In [144]:
#unseen labels-2 by creation of subset 

In [236]:
df = pd.read_csv("tickets.csv")

C:\Users\sridhar\AppData\Local\Temp\ipykernel_3832\1234129387.py:1: DtypeWarning: Columns (10,30,33,34,44,55,57,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("tickets.csv")


### initializing subset size as the model will consider random sample subsets of the dataset and trains on each of the unseen data(here the subsets are the unseen data)

In [237]:
subset_size = 100

### declaring reference columns again

In [242]:
text_columns_2_reference = ['Description', 'Category', 'Priority', 'Affected User Business Unit', 'Department', 'Employee Type', 'Source', 'Status','Affected User Location']

In [243]:
print("the size of subset randomly sampled :",subset_size)

the size of subset randomly sampled : 100


### declaring subset sample

In [244]:
unseen_subset = df.sample(n=subset_size,random_state=42)

In [245]:
unseen_subset['Combined_Text'] = unseen_subset[text_columns_2_reference].apply(lambda row: ' '.join([str(row[col]) for col in text_columns_2_reference]), axis=1)

In [246]:
unseen_subset['Combined_Text'] = unseen_subset['Combined_Text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(str(x))]))

In [247]:
x_unseen_subset = tfidf_vectorizer.transform(unseen_subset['Combined_Text'])

### making prediction on unseen data

In [248]:
y_unseen_prediction = loaded_model.predict(x_unseen_subset)

In [249]:
y_unseen_prediction

array([ 72,  46, 114,  80, 119,  51, 129, 114, 145,  60,  17,   1,   1,
        47,  17,  95,  53,  72,  85,  85, 104,  58,  14,  71,  47, 105,
        63, 114,  26,  71,  43,  53, 117,  55,  17,  89,  47,  58,   1,
         1,  10,  50,  42, 114,  71, 129,  85,  21, 106,   0,  17, 145,
        32,  53,   1,  73,   1,  73,   7,  28,  43, 106,   1,   1,  17,
         1,  55,  26, 135, 145,  43,  92, 121,  15,  32,  91,  55,  61,
        49, 106, 106,  71,   1, 114,  31,  53,  72,  91,  42, 106, 144,
       104,   1,   0,  18, 101,   5, 140,  14,  75], dtype=int64)

In [250]:
y_unseen_prediction_original = label_encoder.inverse_transform(y_unseen_prediction)

In [251]:
y_unseen_prediction_original

array(['GMC - Business Apps', 'EBS - Manufacturing-Supplier EDI',
       'No Group', 'GMC - Plant Floor PLT6', 'PKMF - IT Site Support',
       'EGMF - IT Site Support', 'SCMF - IT Site Support', 'No Group',
       'ZGMF - IT Site Support', 'ENG - NVH Support',
       'APP - ITSM Tool Support', 'AAM - Service Desk',
       'AAM - Service Desk', 'EBS - Order Mgmt-Shipping-Customer EDI',
       'APP - ITSM Tool Support', 'INF - IT Storage',
       'EMMF - IT Site Support', 'GMC - Business Apps',
       'HGMF - IT Site Support', 'HGMF - IT Site Support',
       'India - L2 Service Desk', 'ENG - Electronic Control Systems',
       'APP - IPF Support', 'Freshservice Bundle',
       'EBS - Order Mgmt-Shipping-Customer EDI', 'India - Service Desk',
       'ENG - Rational Tools Support', 'No Group', 'APP - SCADA Support',
       'Freshservice Bundle', 'DBA - Oracle Support',
       'EMMF - IT Site Support', 'OMF - IT Site Support', 'ENG - CAE-FEA',
       'APP - ITSM Tool Support', 'INF - Acti

In [252]:
accuracy_unseen_subset = accuracy_score(unseen_subset['Group'], y_unseen_prediction_original)

###accuracy of the unseen data testing

In [253]:
accuracy_unseen_subset

0.85